In [1]:
pip install kafka-python


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyspark


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pyspark

In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__} pyspark-shell'
os.environ['SPARK_SUBMIT_OPTS'] = '-Djdk.security.auth.login.Config=ignore'

In [5]:
KAFKA_BROKER_URL = "localhost:9092"
KAFKA_TOPIC = "dev-rt_electric_cars"

In [6]:
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, expr
from pyspark.sql.types import StructType, StringType, IntegerType

In [7]:
spark = SparkSession.builder \
    .appName("PySpark-jupyter-streaming-electric-cars") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__}") \
    .config("spark.sql.streaming.checkpointLocation", "./checkpoint") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/11 15:09:06 WARN Utils: Your hostname, h-MacBook-Pro-sl-Gal.local, resolves to a loopback address: 127.0.0.1; using 192.168.167.55 instead (on interface en0)
26/01/11 15:09:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/galrubinstein/.venv3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/galrubinstein/.ivy2.5.2/cache
The jars for the packages stored in: /Users/galrubinstein/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ae5f4ab0-e1f8-403e-9fc5-3583a73a5666;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.1.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.1.0 in central
	found org.apache.kafka#k

In [8]:
kafka_df = spark.readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER_URL) \
  .option("subscribe", KAFKA_TOPIC) \
  .option("startingOffsets", "earliest") \
  .load()

In [9]:
schema = StructType() \
    .add("VIN (1-10)", StringType()) \
    .add("County", StringType()) \
    .add("City", StringType()) \
    .add("State", StringType()) \
    .add("Model Year", StringType()) \
    .add("Make", StringType()) \
    .add("Model", StringType())

# Transform data to dataframe of json format
parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

# Drop rows that contain ANY null values
# This ensures that if a cell value is missing, the row is discarded
clean_df = parsed_df.dropna()

# Transform data to dataframe of json format and calculate the desired result
result_df = clean_df.filter(col("Model Year") == '2023') \
    .groupBy("City", "County", "State") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(3)

In [10]:
result_df.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("res") \
    .start()

26/01/11 15:09:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
26/01/11 15:09:23 WARN MicroBatchExecution: Disabling AQE since AQE is not supported in stateful workloads.


In [11]:
spark.sql("select * from res").show()

+--------+------+-----+-----+
|    City|County|State|count|
+--------+------+-----+-----+
| Seattle|  King|   WA| 5610|
|Bellevue|  King|   WA| 2088|
| Tukwila|  King|   WA| 1773|
+--------+------+-----+-----+

